<h2>File paths and imports<h2>

In [1]:
import sys 
sys.path.append("C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/ChimpRec/Code")

from chimplib.imports import *
from chimplib.recognition_utils import *

#Path to the root of the ChimpRec database ".../Chimprec Dataset/Datasets/Face Recognition/ChimpRec"
chimprec_dataset_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Datasets/Face Recognition/ChimpRec"
chimprec_train_dataset = f"{chimprec_dataset_path}/train"
chimprec_val_dataset = f"{chimprec_dataset_path}/val"
chimprec_test_dataset = f"{chimprec_dataset_path}/test"
#Path to the root of the ChimpRec model ".../Chimprec Dataset/Models/recognition/ChimpRec"
chimprec_models_tl_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Models/recognition/ChimpRec/facenet_16_layers_triplet_loss.pth"

#Path to the root of the CCR database ".../Chimprec Dataset/Datasets/Face Recognition/CCR"
ccr_dataset_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Datasets/Face Recognition/CCR"
ccr_train_dataset = f"{ccr_dataset_path}/train"
ccr_val_dataset = f"{ccr_dataset_path}/val"
ccr_test_dataset = f"{ccr_dataset_path}/test"
#Path to the root of the CCR model ".../Chimprec Dataset/Models/recognition/CCR"
#ccr_models_tl_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Models/recognition/CCR/facenet_13_layers_triplet_loss.pth"
ccr_models_tl_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/ChimpRec/Code/Recognition/FaceNet/Triplet_loss/facenet_13_layers_triplet_loss.pth"

c:\Users\julie\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
c:\Users\julie\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h2>Utils<h2>

In [2]:
#Transformation to be applied to network input images
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# @inputs:
# model_path: path to the saved model file (.pth file) containing weights
# @output:
# model: FaceNet model loaded with the specified weights
def load_model(model_path):
    model = InceptionResnetV1(pretrained='vggface2').eval()
    model.load_state_dict(torch.load(model_path, map_location=device))
    return model.to(device)

In [4]:
# @inputs:
# train_path: path to the training dataset
# val_path: path to the validation dataset
# model: the model used to compute embeddings
# transform: set of transformations to apply to images (resize, normalize, ...) with torchvision.transforms
# has_face_labels: boolean indicating whether face labels (.txt files) are used for cropping
# @output:
# all_embeddings: dictionary {class_name: [embedding vectors]} with embeddings of the validation and train set images linked to their class
def merge_all_embeddings(train_path, val_path, model, transform, has_face_labels):
    emb1 = get_all_embeddings(train_path, model, device, transform, has_face_labels)
    emb2 = get_all_embeddings(val_path, model, device, transform, has_face_labels)
    all_embeddings = {}
    for key in emb1.keys():
        all_embeddings[key] = emb1.get(key, []) + emb2.get(key, [])
    return all_embeddings

In [ ]:
# @inputs:
# test_dataset: path to the test dataset
# model: model used for prediction
# all_embeddings: dictionary {class_name: [embedding vectors]} from the validation and train set for nearest neighbor comparison
# label_fn: function that extracts the ground-truth identity from the image filename
# @output:
# accuracy: float value representing the proportion of correctly classified images in the test dataset
def evaluate_model(test_dataset, model, all_embeddings, label_fn):
    correctly_classified = 0
    total_img = 0

    for img in os.listdir(test_dataset):
        img_path = os.path.join(test_dataset, img)
        image = Image.open(img_path).convert("RGB")

        predicted_label = predict_face_with_tl(image, model, all_embeddings, device, transform, 1)
        real_label = label_fn(img)
        if predicted_label == real_label:
            correctly_classified += 1
        total_img += 1

    return correctly_classified / total_img

<h2>Evaluation on the test set<h2>

<h3>ChimpRec<h3>

In [6]:
#To switch from the file name (for example: “AD1.jpg”) to its identity (for example: “AD”)
chimprec_label_fn = lambda img: img[:2] 

chimprec_model = load_model(chimprec_models_tl_path)
all_embeddings = merge_all_embeddings(chimprec_train_dataset, chimprec_val_dataset, chimprec_model, transform, True)
accuracy = evaluate_model(chimprec_test_dataset, chimprec_model, all_embeddings, chimprec_label_fn)
print("ChimpRec Accuracy:", accuracy)

PD
AD
PD
AD
IV
AD
PD
AD
PD
AD
AD
AD
BL
AD
LM
BL
PD
BL
PD
BL
PD
BL
PD
BL
LM
BL
KG
BL
MM
BS
PD
BS
MM
BS
MM
BS
AD
BS
KR
BS
PD
BS
NJ
BS
KR
DK
KR
DK
KR
DK
LM
DK
PD
DK
LM
DK
DK
DK
IV
IV
IV
IV
LM
IV
PD
IV
LM
IV
LM
IV
LM
IV
MG
JJ
MG
JJ
MK
JJ
MK
JJ
PD
JJ
PD
JJ
PD
JJ
LM
JJ
AD
KG
AD
KG
AD
KG
AD
KG
AD
KG
NJ
KM
NJ
KM
AD
KM
PD
KM
PD
KM
PD
KM
LM
KM
PD
LM
LM
LM
PD
LM
PD
LM
PD
LM
PD
LM
TT
LM
LM
LM
TT
LM
IV
MG
LM
MG
LM
MG
MG
MG
MG
MG
MG
MG
DK
MG
LM
MG
PD
MG
AD
MK
AD
MK
AD
MK
LM
MK
LM
MK
LM
MZ
LM
MZ
LM
MZ
LM
MZ
LM
MZ
LM
MZ
LM
MZ
PD
MZ
DK
NJ
LM
NJ
LM
NJ
DK
NJ
PD
NJ
PD
NJ
PD
NJ
PD
NJ
LM
NJ
LM
NR
MM
NR
MZ
NR
NR
NR
MZ
NR
MM
NR
MM
NR
LM
PD
PD
PD
PD
PD
PD
PD
LM
PD
PD
PD
KR
PD
LM
TC
PD
TC
LM
TC
TC
TC
PD
TC
MM
TC
TT
TC
LM
TS
LM
TS
PD
TS
LM
TS
PD
TS
MK
TS
MK
TS
LM
TT
LM
TT
LM
TT
TT
TT
LM
TT
LM
TT
LM
TT
PD
TT
ChimpRec Accuracy: 0.12121212121212122


<h3>CCR<h3>

In [6]:
#To switch from the file name (for example: “FANA_0.jpg”) to its identity (for example: “FANA”)
ccr_label_fn = lambda img: img.split("_")[0]

ccr_model = load_model(ccr_models_tl_path)
all_embeddings = merge_all_embeddings(ccr_train_dataset, ccr_val_dataset, ccr_model, transform, False)
accuracy = evaluate_model(ccr_test_dataset, ccr_model, all_embeddings, ccr_label_fn)
print("CCR Accuracy:", accuracy)

CCR Accuracy: 0.6304
